In [1]:
import numpy as np 
import h5py
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
with h5py.File('full_dataset_vectors.h5', 'r') as dataset:
    xtrain, xtest = dataset["X_train"][:], dataset["X_test"][:]
    ytrain, ytest = dataset["y_train"][:], dataset["y_test"][:]
    xtrain = np.array(xtrain)
    xtest = np.array(xtest)
    print('train shape:', xtrain.shape)
    print('test shape:', xtest.shape)
    xtrain = xtrain.reshape(xtrain.shape[0], 16, 16, 16, 1)
    xtest = xtest.reshape(xtest.shape[0], 16, 16, 16, 1)
    ytrain, ytest = to_categorical(ytrain, 10), to_categorical(ytest, 10)

train shape: (10000, 4096)
test shape: (2000, 4096)


In [3]:
model = Sequential()
model.add(layers.Conv3D(32,(3,3,3),activation='relu',input_shape=(16,16,16,1),bias_initializer=Constant(0.01)))
model.add(layers.Conv3D(32,(3,3,3),activation='relu',bias_initializer=Constant(0.01)))
model.add(layers.MaxPooling3D((2,2,2)))
model.add(layers.Conv3D(64,(3,3,3),activation='relu'))
model.add(layers.Conv3D(64,(2,2,2),activation='relu'))
model.add(layers.MaxPooling3D((2,2,2)))
model.add(layers.Dropout(0.6))
model.add(layers.Flatten())
model.add(layers.Dense(256,'relu'))
model.add(layers.Dropout(0.7))
model.add(layers.Dense(128,'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10,'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 14, 14, 14, 32)    896       
                                                                 
 conv3d_1 (Conv3D)           (None, 12, 12, 12, 32)    27680     
                                                                 
 max_pooling3d (MaxPooling3D  (None, 6, 6, 6, 32)      0         
 )                                                               
                                                                 
 conv3d_2 (Conv3D)           (None, 4, 4, 4, 64)       55360     
                                                                 
 conv3d_3 (Conv3D)           (None, 3, 3, 3, 64)       32832     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 1, 1, 1, 64)      0         
 3D)                                                    

In [4]:
model.compile(Adam(0.001),'categorical_crossentropy',['accuracy'])
model.fit(xtrain,ytrain,epochs=20,batch_size=32,verbose=1,validation_data=(xtest,ytest),callbacks=[EarlyStopping(patience=15)])


Epoch 1/20
313/313 [==============================] - 153s 487ms/step - loss: 2.2475 - accuracy: 0.1455 - val_loss: 2.1203 - val_accuracy: 0.2500
Epoch 2/20
313/313 [==============================] - 154s 491ms/step - loss: 2.0617 - accuracy: 0.2484 - val_loss: 1.8675 - val_accuracy: 0.3165
Epoch 3/20
313/313 [==============================] - 152s 487ms/step - loss: 1.8868 - accuracy: 0.3159 - val_loss: 1.6531 - val_accuracy: 0.4040
Epoch 4/20
313/313 [==============================] - 165s 527ms/step - loss: 1.7421 - accuracy: 0.3688 - val_loss: 1.5162 - val_accuracy: 0.4435
Epoch 5/20
313/313 [==============================] - 150s 479ms/step - loss: 1.6176 - accuracy: 0.4162 - val_loss: 1.5313 - val_accuracy: 0.4360
Epoch 6/20
313/313 [==============================] - 159s 508ms/step - loss: 1.5277 - accuracy: 0.4680 - val_loss: 1.3375 - val_accuracy: 0.5455
Epoch 7/20
313/313 [==============================] - 182s 581ms/step - loss: 1.4624 - accuracy: 0.4911 - val_loss: 1.2347 -

In [5]:
_, acc = model.evaluate(xtrain, ytrain)
print('training accuracy:', str(round(acc*100, 2))+'%')
_, acc = model.evaluate(xtest, ytest)
print('testing accuracy:', str(round(acc*100, 2))+'%')

313/313 [==============================] - 23s 73ms/step - loss: 0.8861 - accuracy: 0.6904
training accuracy: 69.04%
63/63 [==============================] - 5s 77ms/step - loss: 1.0632 - accuracy: 0.6475
testing accuracy: 64.75%
